In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib as plt 

In [2]:
#CSV data files
gdp = pd.read_csv('gdp.CSV')
gdp_growth = pd.read_csv('gdp_growth.csv')
gpd_per_capita = pd.read_csv('gdp_per_capita.csv')
gdp_per_capita_growth = pd.read_csv('gdp_per_capita_growth.csv')
Chinese_pop = pd.read_csv('Chinas Population En.csv')
Japanese_pop =pd.read_csv('japanese_census.csv')
Fertility_rate = pd.read_csv('API_SP.DYN.TFRT.IN_DS2_EN_csv_v2_4023233.csv')
Age_14under = pd.read_csv('API_SP.POP.0014.TO.ZS_DS2_en_csv_v2_4153151.csv')
Age_dependency= pd.read_csv('API_SP.POP.DPND_DS2_en_csv_v2_4154688.csv')
Age_15to64 = pd.read_csv('API_SP.POP.1564.TO.ZS_DS2_en_csv_v2_4150831.csv')
Age_64above = pd.read_csv('API_SP.POP.65UP.TO.ZS_DS2_en_csv_v2_4154737.csv')

In [3]:
#Japanese population data prefecture
Japanese_pop.columns
Japanese_pop['Prefecture'].unique()
Japanese_pop_prefect = Japanese_pop.set_index(['Year', 'Prefecture',])
Japanese_pop_prefect.drop("Unnamed: 0",axis=1, inplace= True)
Years = range(1980, 2010, 5)
JP_Prefecture_Pop = Japanese_pop_prefect.reset_index()
JP_Prefecture_Pop['Year'] = JP_Prefecture_Pop['Year'].astype(str)
JP_Prefecture_Pop


,Year,Prefecture,Age Lower Bound,Age Upper Bound,Gender,Population
0,1920,Hokkaido,0,4.0,Male,190044
1,1920,Hokkaido,0,4.0,Female,186632
2,1920,Hokkaido,5,9.0,Male,161814
3,1920,Hokkaido,5,9.0,Female,157556
4,1920,Hokkaido,10,14.0,Male,138018
...,...,...,...,...,...,...
33449,2015,Okinawa Prefecture,75,79.0,Female,30607
33450,2015,Okinawa Prefecture,80,84.0,Male,18017
33451,2015,Okinawa Prefecture,80,84.0,Female,25182
33452,2015,Okinawa Prefecture,85,NaN,Male,12637


In [4]:
#Total population of Japan
JP_Total_pop = pd.DataFrame(Japanese_pop_prefect.groupby('Year')['Population'].sum())
JP_Total_pop.reset_index(inplace=True)
JP_Total_pop['Year'] =JP_Total_pop['Year'].astype(str)
JP_Total_pop


,Year,Population
0,1920,55963053
1,1925,59736772
2,1930,64450005
3,1935,69254148
4,1940,73074109
5,1945,72324729
6,1950,84101821
7,1955,90075711
8,1960,94301494
9,1965,99209086


In [5]:
#Cleaning Japan's GDP per capita data 
Japan_gdp_per_capita = gpd_per_capita.set_index("Country Name").loc["Japan"]
JP_gppp = pd.DataFrame(Japan_gdp_per_capita )
JP_gppp.reset_index(inplace=True)
JP_gppp.rename(columns = {'index':'Year'}, inplace = True)
JP_gppp.drop(0, axis = 0, inplace = True)

In [6]:
#Japan' GDP per Capita growth
Japan_gdpp_growth = gdp_per_capita_growth.set_index("Country Name").loc["Japan"]
JP_gg= pd.DataFrame(Japan_gdpp_growth)
JP_gg.reset_index(inplace=True)
JP_gg.rename(columns = {'index':'Year'}, inplace = True)
JP_gg.drop(0, axis = 0, inplace = True)

In [7]:
#Merge Japan's GDP per Capita Growth and GDP per Capita
JP_GDP_Overall_data = pd.merge(JP_gppp, JP_gg, how = 'inner', on ='Year')
JP_GDP_Overall_data.rename(columns = {'Japan_x':'GDP Per Capita', 'Japan_y': "GDP Per Capita Growth Rate"}, inplace = True)
JP_GDP_Overall_data.dropna(inplace = True)
JP_GDP_Overall_data
JP_GDP_Overall_data.to_csv('JP_GDP_Overall_data.csv')

In [8]:
#GDP data with population
JP_data = pd.merge(JP_Total_pop, JP_GDP_Overall_data, how = 'inner', on = 'Year')
JP_data.dtypes
JP_data

,Year,Population,GDP Per Capita,GDP Per Capita Growth Rate
0,1965,99209086,928.518849,4.68645
1,1970,104665171,2056.122046,1.339145
2,1975,111893438,4674.445481,1.787836
3,1980,116989033,9463.353855,2.010416
4,1985,121007577,11576.692113,4.576745
5,1990,123284810,25371.464171,4.545274
6,1995,125439273,44197.619101,2.39052
7,2000,126697282,39169.35957,2.592891
8,2005,127285653,37812.89502,1.79434
9,2010,127080929,44968.156235,4.079223


In [9]:
#Data on Japanese population grouped by year, age and ignroign the prefecture
JP_Prefecture_data =  pd.merge(JP_Prefecture_Pop, JP_GDP_Overall_data, how = 'inner', on = 'Year')
JP_Prefecture_data.set_index('Year', inplace = True)
JP_demo = JP_Prefecture_data.groupby(['Year','Gender','Age Lower Bound', 'Age Upper Bound']).sum()
JP_demo.to_csv('JP_demo.csv')

In [10]:
#Data on Chinese population
Chinese_pop.rename(columns={'Unnamed: 0':"Year"}, inplace = True)
Chinese_pop.set_index('Year', inplace = True)
Chinese_pop.to_csv('CN_demo.csv')

In [11]:
#Chinese GDP Per Capita
China_gdp_per_capita = gpd_per_capita.set_index("Country Name").loc["China"]
CN_gppp = pd.DataFrame(China_gdp_per_capita )
CN_gppp.reset_index(inplace=True)
CN_gppp.rename(columns = {'index':'Year'}, inplace = True)
CN_gppp.drop(0, axis = 0, inplace = True)
CN_gppp.dropna(inplace = True)
CN_gppp

,Year,China
1,1960,89.520542
2,1961,75.805838
3,1962,70.909412
4,1963,74.313643
5,1964,85.498555
...,...,...
57,2016,8094.363367
58,2017,8816.986905
59,2018,9905.342004
60,2019,10143.838172


In [12]:
#China's GDP per Capita growth
China_gdp_growth = gdp_per_capita_growth.set_index("Country Name").loc["China"]
CN_gg= pd.DataFrame(China_gdp_growth)
CN_gg.reset_index(inplace=True)
CN_gg.rename(columns = {'index':'Year'}, inplace = True)
CN_gg.drop(0, axis = 0, inplace = True)

In [13]:
CN_GDP_Overall_data = pd.merge(CN_gppp, CN_gg, how = 'inner', on ='Year')
CN_GDP_Overall_data.rename(columns = {'China_x':'GDP Per Capita', 'China': "GDP Per Capita Growth Rate"}, inplace = True)
CN_GDP_Overall_data.dropna()
CN_GDP_Overall_data.to_csv('CN_GDP_Overall_data.csv')

In [14]:
#Cleaning up the Fertility rate 
F_rate = Fertility_rate[3:] 
F_rate.columns = F_rate.iloc[0]
F = F_rate[1:-1]
F.set_index('Country Name',inplace = True)
F_rate_CN_JP  = F.loc[['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia']]
F_rate_CN_JP.drop(columns = ['Indicator Name', 'Indicator Code'], inplace = True)
F_rate_CN_JP_T = F_rate_CN_JP.T
F_rate_CN_JP_T.reset_index(inplace = True)
F_rate_CN_JP_T.columns = ["Year", 'China Fertility rate, total (births per woman)', \
                           'Japan Fertility rate, total (births per woman)',\
                          'Korea Fertility rate, total (births per woman)', \
                          'Singapore Fertility rate, total (births per woman)',\
                          'Malaysia Fertility rate, total (births per woman)' ]
F_rate_CN_JP_T1 = F_rate_CN_JP_T.loc[1:]
F_rate_CN_JP_T1['Year'] = F_rate_CN_JP_T1['Year'].astype(int)
F_rate_CN_JP_T1

C:\Users\sangr\AppData\Local\Temp/ipykernel_17000/2698940517.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  F_rate_CN_JP_T1['Year'] = F_rate_CN_JP_T1['Year'].astype(int)


,Year,"China Fertility rate, total (births per woman)","Japan Fertility rate, total (births per woman)","Korea Fertility rate, total (births per woman)","Singapore Fertility rate, total (births per woman)","Malaysia Fertility rate, total (births per woman)"
1,1960,5.756,2.001,6.095,5.76,6.45
2,1961,5.905,2.05,5.929,5.41,6.42
3,1962,6.062,2.01,5.738,5.21,6.353
4,1963,6.206,2.02,5.533,5.16,6.246
5,1964,6.32,2.05,5.325,4.97,6.1
...,...,...,...,...,...,...
57,2016,1.675,1.44,1.172,1.2,2.037
58,2017,1.683,1.43,1.052,1.16,2.019
59,2018,1.69,1.42,0.977,1.14,2.001
60,2019,1.696,1.36,0.918,1.14,1.983


In [15]:
#Global average F rate
avg = F.mean()
gavg= pd.DataFrame(avg)
gavg.reset_index(inplace = True)
gavg.columns = ['Year', 'Fertility Rate']
gavg['Year'] = gavg['Year'].astype(int)
gavg

C:\Users\sangr\AppData\Local\Temp/ipykernel_17000/3174611944.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  avg = F.mean()


,Year,Fertility Rate
0,1960,5.505751
1,1961,5.489795
2,1962,5.490831
3,1963,5.489791
4,1964,5.452046
...,...,...
56,2016,2.747591
57,2017,2.707341
58,2018,2.676643
59,2019,2.642967


In [16]:
#Merge the fertility of CN and JP with global average dataframe
F_Rate_CN_JP_Global_Avg = pd.merge(F_rate_CN_JP_T1, gavg, how = 'left', on = 'Year')

In [17]:
F_Rate_CN_JP_Global_Avg.set_index('Year', inplace = True)

In [18]:
F_Rate_CN_JP_Global_Avg.to_csv('Fertility_rate.csv')

In [19]:
#Age depedency ratio between the two countries
Age_dependency.columns =  Age_dependency.iloc[3]
Age_dependency_CN_JP = Age_dependency.set_index('Country Name').loc[['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia']]
Age_dependency_CN_JP.drop(columns = ['Indicator Name', 'Indicator Code'], inplace = True)
Age_dependency_CN_JP_T = Age_dependency_CN_JP.T
Age_dependency_CN_JP_T.reset_index(inplace = True)
Age_dependency_CN_JP_T.columns = ["Year", 'China', \
                           'Japan',\
                          'Korea', \
                          'Singapore',\
                          'Malaysia' ]
Age_dependency_D = Age_dependency_CN_JP_T.loc[1:]
Age_dependency_D['Year'] = Age_dependency_D['Year'].astype(int)
Age_dependency_DF = Age_dependency_D.set_index('Year')
Age_dependency_DF

C:\Users\sangr\AppData\Local\Temp/ipykernel_17000/2592434842.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Age_dependency_D['Year'] = Age_dependency_D['Year'].astype(int)


,China,Japan,Korea,Singapore,Malaysia
Year,,,,,
1960,77.02468,55.969004,87.284542,82.725727,95.736596
1961,77.260002,54.173853,88.908117,85.471474,96.615217
1962,78.433757,52.24188,89.427848,87.10392,97.526358
1963,79.877719,50.299641,89.17725,87.589222,98.189332
1964,80.673601,48.557168,88.687898,87.174635,98.261781
...,...,...,...,...,...
2016,38.498875,65.306132,36.679002,28.300106,44.758403
2017,39.435946,66.455463,37.13235,29.520864,44.462208
2018,40.445269,67.42908,37.725639,31.133213,44.231246


In [20]:
#Global average F rate
D_rate = Age_dependency[3:] 
D_rate.columns = D_rate.iloc[0]
D =D_rate[1:-1]
avg = D.mean()
gavg= pd.DataFrame(avg)
gavg.reset_index(inplace = True)
gavg.columns = ['Year', 'D Rate']
gavg['Year'] = gavg['Year'].astype(int)
gavg

C:\Users\sangr\AppData\Local\Temp/ipykernel_17000/4207096119.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  avg = D.mean()


,Year,D Rate
0,1960,78.975530
1,1961,79.864276
2,1962,80.640761
3,1963,81.275930
4,1964,81.689391
...,...,...
56,2016,58.931652
57,2017,58.907094
58,2018,58.895158
59,2019,58.875131


In [21]:
#Merge the depedency rate of  CN and JP with global average dataframe
D_Rate_CN_JP_Global_Avg = pd.merge(Age_dependency_DF, gavg, how = 'left', on = 'Year')

In [22]:
D_Rate_CN_JP_Global_Avg.to_csv('age_dependency.csv')

In [23]:
#under Age 14 population of each country
Age_14under.columns =  Age_14under.iloc[3]
Age_14under_CN_JP = Age_14under.set_index('Country Name').loc[['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia' ]]
Age_14under_CN_JP.drop(columns = ['Indicator Name', 'Indicator Code'], inplace = True)
Age_14under_CN_JP_T = Age_14under_CN_JP.T
Age_14under_CN_JP_T.reset_index(inplace = True)
Age_14under_CN_JP_T.columns = ['Year', 'China', 'Japan', 'Korea', 'Singapore', 'Malaysia']
Age_14under_D = Age_14under_CN_JP_T.loc[1:]
Age_14under_D['Year'] = Age_14under_D['Year'].astype(int)
Age_14under_DF = Age_14under_D.set_index('Year')
Age_14under_DF

C:\Users\sangr\AppData\Local\Temp/ipykernel_17000/4089913500.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Age_14under_D['Year'] = Age_14under_D['Year'].astype(int)


,China,Japan,Korea,Singapore,Malaysia
Year,,,,,
1960,39.822192,30.2639,43.238846,43.228586,45.49392
1961,39.874431,29.409036,43.670999,43.94335,45.767243
1962,40.282471,28.478359,43.779999,44.299726,46.041124
1963,40.807495,27.520804,43.676053,44.31299,46.248819
1964,41.139201,26.630508,43.524095,44.063062,46.311197
...,...,...,...,...,...
2016,17.994616,12.91436,13.497931,12.289246,24.716277
2017,17.937361,12.81439,13.225499,12.212456,24.343791
2018,17.877003,12.696849,12.973324,12.278283,23.995141


In [25]:
Age_64above.columns =  Age_64above.iloc[3]
Age_64above_1 = Age_64above.set_index('Country Name').loc[['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia' ]]
Age_64above_1.drop(columns = ['Indicator Name', 'Indicator Code'], inplace = True)
Age_64above_T = Age_64above_1.T
Age_64above_T.reset_index(inplace = True)
Age_64above_T.columns = ['Year', 'China', 'Japan', 'Korea', 'Singapore', 'Malaysia']
Age_64above_D = Age_64above_T.loc[1:]
Age_64above_D['Year'] = Age_64above_D['Year'].astype(int)
Age_64above_DF = Age_64above_D.set_index('Year')

C:\Users\sangr\AppData\Local\Temp/ipykernel_17000/3129128198.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Age_64above_D['Year'] = Age_64above_D['Year'].astype(int)


In [27]:
Age_15to64.columns =  Age_15to64.iloc[3]
Age_15to64_1 = Age_15to64.set_index('Country Name').loc[['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia' ]]
Age_15to64_1.drop(columns = ['Indicator Name', 'Indicator Code'], inplace = True)
Age_15to64_T = Age_15to64_1.T
Age_15to64_T.reset_index(inplace = True)
Age_15to64_T.columns = ['Year', 'China', 'Japan', 'Korea', 'Singapore', 'Malaysia']
Age_15to64_D = Age_15to64_T.loc[1:]
Age_15to64_D['Year'] = Age_15to64_D['Year'].astype(int)
Age_15to64_DF = Age_15to64_D.set_index('Year')

C:\Users\sangr\AppData\Local\Temp/ipykernel_17000/1449979569.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Age_15to64_D['Year'] = Age_15to64_D['Year'].astype(int)


In [29]:
#GDP of each country
gdp_country = gdp.set_index('Country Name').loc[['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia' ]].reset_index().T
gdp_country.columns =  ['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia' ]
gdp_1= gdp_country.iloc[2:-1]
gdp_df = gdp_1.reset_index().rename(columns = {'index':'Year'})
gdp_df['Year'] = gdp_df['Year'].astype(int)
gdp_df.set_index('Year', inplace = True)

In [41]:
#GDP growth
gdp_growth_country = gdp_growth.set_index('Country Name').loc[['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia' ]].reset_index().T
gdp_growth_country.columns =  ['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia' ]
gdp_growth_1= gdp_growth_country.iloc[2:-1]
gdp_growth_df = gdp_growth_1.reset_index().rename(columns = {'index':'Year'})
gdp_growth_df['Year'] = gdp_growth_df['Year'].astype(int)
gdp_growth_df.set_index('Year', inplace = True)
gdp_growth_df

,China,Japan,"Korea, Rep.",Singapore,Malaysia
Year,,,,,
1960,NaN,NaN,NaN,NaN,NaN
1961,-27.27,12.043536,6.935993,8.136888,7.597994
1962,-5.58,8.908973,3.895273,7.554046,6.42103
1963,10.3,8.473642,9.020568,10.040621,7.338804
1964,18.18,11.676708,9.473825,-3.103333,5.358963
...,...,...,...,...,...
2016,6.848762,0.753827,2.946882,3.329034,4.449781
2017,6.947201,1.675332,3.159636,4.520379,5.812722
2018,6.749774,0.558851,2.907404,3.49747,4.843057


In [31]:
#GDP per capita
gpd_per_capita
gpd_per_capita_country = gpd_per_capita.set_index('Country Name').loc[['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia' ]].reset_index().T
gpd_per_capita_country.columns =  ['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia' ]
gpd_per_capita_1= gpd_per_capita_country.iloc[2:-1]
gpd_per_capita_df = gpd_per_capita_1.reset_index().rename(columns = {'index':'Year'})
gpd_per_capita_df['Year'] = gpd_per_capita_df['Year'].astype(int)
gpd_per_capita_df.set_index('Year', inplace = True)

In [32]:
#GDP per capita growth
gdp_per_capita_growth
gdp_per_capita_growth_country = gdp_per_capita_growth.set_index('Country Name').loc[['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia' ]].reset_index().T
gdp_per_capita_growth_country.columns =  ['China', 'Japan', 'Korea, Rep.', 'Singapore', 'Malaysia' ]
gdp_per_capita_growth_1= gdp_per_capita_growth_country.iloc[2:-1]
gdp_per_capita_growth_df = gdp_per_capita_growth_1.reset_index().rename(columns = {'index':'Year'})
gdp_per_capita_growth_df['Year'] = gdp_per_capita_growth_df['Year'].astype(int)
gdp_per_capita_growth_df.set_index('Year', inplace = True)

In [72]:
Japan = pd.DataFrame()
Japan['GDP'] = gdp_df['Japan']
Japan['Fertility rate, total\
 (births per woman)'] = F_Rate_CN_JP_Global_Avg['Japan Fertility rate, total (births per woman)']
Japan['GDP Growth']= gdp_growth_df['Japan']
Japan['GDP Per Capita'] = gpd_per_capita_df['Japan']
Japan['GDP Per Capita Growth'] = gdp_per_capita_growth_df['Japan']
Japan['14 Under'] = Age_14under_DF['Japan']
Japan['15 to 64'] = Age_15to64_DF['Japan']
Japan['64 above'] = Age_64above_DF['Japan']
Japan['Work Population(%)'] = Age_dependency_DF['Japan']
Japan.to_csv('Japan.csv')

In [73]:
China = pd.DataFrame()
China['GDP'] = gdp_df['China']
China['Fertility rate, total\
 (births per woman)'] = F_Rate_CN_JP_Global_Avg['China Fertility rate, total (births per woman)']
China['GDP Growth']= gdp_growth_df['China']
China['GDP Per Capita'] = gpd_per_capita_df['China']
China['GDP Per Capita Growth'] = gdp_per_capita_growth_df['China']
China['14 Under'] = Age_14under_DF['China']
China['15 to 64'] = Age_15to64_DF['China']
China['64 above'] = Age_64above_DF['China']
China['Work Population(%)'] = Age_dependency_DF['China']
China.to_csv('China.csv')

In [74]:
Korea = pd.DataFrame()
Korea['GDP'] = gdp_df['Korea, Rep.']
Korea['Fertility rate, total\
 (births per woman)'] = F_Rate_CN_JP_Global_Avg['Korea Fertility rate, total (births per woman)']
Korea['GDP Growth']= gdp_growth_df['Korea, Rep.']
Korea['GDP Per Capita'] = gpd_per_capita_df['Korea, Rep.']
Korea['GDP Per Capita Growth'] = gdp_per_capita_growth_df['Korea, Rep.']
Korea['14 Under'] = Age_14under_DF['Korea']
Korea['15 to 64'] = Age_15to64_DF['Korea']
Korea['64 above'] = Age_64above_DF['Korea']
Korea['Work Population(%)'] = Age_dependency_DF['Korea']
Korea.to_csv('Korea.csv')

In [75]:
Singapore = pd.DataFrame()
Singapore['GDP'] = gdp_df['Singapore']
Singapore['Fertility rate, total\
 (births per woman)'] = F_Rate_CN_JP_Global_Avg['Singapore Fertility rate, total (births per woman)']
Singapore['GDP Growth']= gdp_growth_df['Singapore']
Singapore['GDP Per Capita'] = gpd_per_capita_df['Singapore']
Singapore['GDP Per Capita Growth'] = gdp_per_capita_growth_df['Singapore']
Singapore['14 Under'] = Age_14under_DF['Singapore']
Singapore['15 to 64'] = Age_15to64_DF['Singapore']
Singapore['64 above'] = Age_64above_DF['Singapore']
Singapore['Work Population(%)'] = Age_dependency_DF['Singapore']
Singapore.to_csv('Singapore.csv')

In [76]:
Malaysia = pd.DataFrame()
Malaysia['GDP'] = gdp_df['Malaysia']
Malaysia['Fertility rate, total\
 (births per woman)'] = F_Rate_CN_JP_Global_Avg['Malaysia Fertility rate, total (births per woman)']
Malaysia['GDP Growth']= gdp_growth_df['Malaysia']
Malaysia['GDP Per Capita'] = gpd_per_capita_df['Malaysia']
Malaysia['GDP Per Capita Growth'] = gdp_per_capita_growth_df['Malaysia']
Malaysia['14 Under'] = Age_14under_DF['Malaysia']
Malaysia['15 to 64'] = Age_15to64_DF['Malaysia']
Malaysia['64 above'] = Age_64above_DF['Malaysia']
Malaysia['Work Population(%)'] = Age_dependency_DF['Malaysia']
Malaysia.to_csv('Malaysia.csv')